## 챕터3 딕셔너리와 집합

이 장에서는 다음과 같은 내용을 설명한다.
* 공통적으로 사용되는 딕셔너리 메서드
* 없는 키에 대한 특별 처리
* 표준 라이브러리에서 제공하는 다양한 딕녀리 클래스
* set과 frozen형
* 해시 테이블의 작동 방식
* 해시 테이블의 의미(키 자료형 제한, 예측할 수 없는 순서 등)

### 지능형 딕셔너리
- 모든 반복형 객체에서 키-값 쌍을 생성함으로써 딕셔너리 객체를 만들 수 있다.

In [2]:
DIAL_CODES = [(86, 'China'), (91, 'India'), (1, 'United States'), (62, 'Indonesia')]
country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86, 'India': 91, 'United States': 1, 'Indonesia': 62}

In [3]:
{code: country.upper() for country, code in country_code.items() if code<66}

{1: 'UNITED STATES', 62: 'INDONESIA'}

### 존재하지 않는 키를 setdefault()로 처리하기

In [ ]:
"""단어가 나타나는 위치를 가리키는 인덱스를 만든다."""

import sys
import re

WORD_RE = re.compile(r'\w+')
index = {}

with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            #occurences = index.get(word, []) #단어(word)에 대한 occurences 리스트를 가져오거나, 단어가 없으면 빈 배열을 가져온다
            #occurences.append(location) # 새로 만든 location을 occurences에 추가한다.
            #index[word] = occurences # 변경된 occurences를 index 딕셔너리에 넣는다. 그러면 index를 한 번 더 검색한다.
            index.setdefault(word,[]).append(location)
            
# 알파벳순으로 출력한다.
for word in sorted(index, key=str.upper):
    print(word, index[word])

### 융통성 있게 키를 조회하는 매핑
1. defaultdict
2. _missing_()

In [ ]:
"""단어가 나타나는 위치를 가리키는 인덱스를 만든다."""

import sys
import re
import collections

WORD_RE = re.compile(r'\w+')

index = collections.defaultdict(list)
with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            index[word].append(location)
            
# 알파벳순으로 출력한다.
for word in sorted(index, key=str.upper):
    print(word, index[word])

In [ ]:
class StrKeyDict0(dict): # dict를 상속한다.
    
    def __missing__(self, key):
        if isinstance(key, str): # 키가 문자열인지 확인한다. 키가 문자열이고 존재하지 않으면 KeyError가 발생한다.
            raise KeyError(key)
        return self[str(key)] # 키에서 문자열을 만들고 조회한다.
    
    def get(self, key, default=None):
        try:
            return self[key] # get() 메서드는 self[key] 표기법을 이용해서 __getitem__() 메서드에 위임한다. 이렇게 함으로써 __missing__()메서드가 작동할 수 있는 기회를 준다.
        except KeyError:
            return default # KeyError가 발생하면 __missing__() 메서드가 이미 실패한 것이므로 default를 반환한다.
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys() # 수정하지 않은 (문자열이 아닐 수 있는) 키를 검색하고 나서, 키에서 만든 문자열로 검색한다.

### 그 외 매핑형
1. collections.OrderedDict
2. collections.ChainMao
3. collections.Counter
4. collections.UserDict

In [9]:
ct = collections.Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [12]:
ct.update('aaaaazzz') 
ct

Counter({'a': 20, 'b': 2, 'r': 2, 'c': 1, 'd': 1, 'z': 9})

In [13]:
ct.most_common(2)
ct

Counter({'a': 20, 'b': 2, 'r': 2, 'c': 1, 'd': 1, 'z': 9})

### UserDict 상속하기
- UserDict는 dict를 상속하지 않고 내부에 실제 항목을 담고 있는 data라고 하는 dict객체를 갖고 있다.

In [ ]:
import collections

class StrKeyDict(collections.UserDict): # StrKeyDict는 UserDict를 상속한다.
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    # 저장된 키가 모두 str형이므로 StrKeyDict에서 self.keys()를 호출하는 방법과 달리 
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item # __setitem__()메서드는 모든 키를 str형으로 변환하므로, 연산을 self.data에 위임할 때 더 간단히 작성할 수 있다.

- 다음과 같은 메서드는 상당히 유용하다.
    - MutableMapping.update(): 다른 매핑이나 (키, 값)쌍의 반복형 및 키워드 인수에서 객체를 로딩하기 위해 __init__()에 의해 사용될 수도 있다. 이 메서드는 항목을 추가하기 위해 'self[키]=값'구문을 사용하므로 결국 서브클래스에서 구현한 __setitem__()메서드를 호출하게 된다.
    - Mapping.get()
 
### 불변 매핑

In [14]:
# dict에서 읽기 전용 mappingproxy 객체를 생성하는 MappingProxyType
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [15]:
d_proxy[1] # d에 들어있는 항목은 d_proxy를 통해서 볼 수 있다.

'A'

In [16]:
d_proxy[2] = 'x' # d_proxy를 변경할 수 없다.

TypeError: 'mappingproxy' object does not support item assignment

In [17]:
d[2] = 'B'
d_proxy # 동적인 d_proxy는 d에 대한 변경을 바로 반영한다.

mappingproxy({1: 'A', 2: 'B'})

### 집합 이론
- 고유함을 보장하는 것 외에 집합형은 중위 연산자를 이용해서 기본적인 집합 연산을 구현한다.

### 지능형 집합

In [18]:
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}
# 코드 번호가 32에서 255 사이에 있는 문자 중 문자명 안에 'SIGN'단어가 들어 있는 문자들의 집합을 생성한다.

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

### dict와 set의 내부 구조

#### 딕셔너리의 작동 방식
* 다음 요구사항을 모두 만족하는 객체는 해시 가능하다.
    1. 객체의 수명 주기 동안 언제나 동일한 값을 반환하는 _hash_() 메서드를 제공해서 hash() 함수를 지원한다.
    2. _eq_() 메서드를 통해 동치성을 판단할 수 있다.
    3. a == b가 참이면, hash(a) == hash(b)도 반드시 참이어야 한다.
        
* dict가 내부적으로 해시 테이블을 사용하고 있고 해시가 제대로 작동하려면 빈 공간이 충분해야 하므로, dict의 메모리 공간 효율성은 높지 않다.

* 딕셔너리는 메모리 오버헤드가 상당히 크지만, 메모리에 로딩되는 한 딕셔너리 크기와 무관하게 빠른 접근 속도를 제공한다.

* 해시 충돌이 발생하면 두 번째 키는 충돌이 발생하지 않았을 때의 정상적인 위치와 다른 곳에 놓이게 된다. 따라서 dict([(key1, value1), (key2, value2)])로 생성한 딕셔너리와 dict([(key2, value2), (key1, value1)])으로 생성한 딕셔너리는 동일하지만, key1과 key2의 해시가 충돌하면 키의 순서는 달라진다.

* 딕셔너리에 항목을 추가하면 기존 키의 순서가 변경될 수 있다.



#### 집합의 작동 방식 - 현실적으로 미치는 영향
* set 요소는 모두 해시 가능한 객체여야 한다.
* set의 메모리 오버헤드가 상당히 크다.
* 집합에 속해 있는지 매우 효율적으로 검사할 수 있다.
* 요소의 순서는 요소를 추가한 순서에 따라 달라진다.
* 요소를 집합에 추가하면 다른 요소의 순서가 바뀔 수 있다.